In [ ]:
import datetime
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

import pdb

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
warnings.filterwarnings('ignore')

wbpath = 'data/DP Beh Data 2022-23.xlsm'
#read in necessary sheets 
dfcolnmes = pd.read_excel(wbpath, sheet_name='Oct', skiprows=0)
df = pd.read_excel(wbpath, sheet_name='Oct', skiprows=1)

dfinfo = pd.read_excel(wbpath, sheet_name=0)
dfstudet = pd.read_excel(wbpath, sheet_name=2, header=None)

#get years variable 
year = dfinfo.iloc[3,1]
year1, year2 = str.split(year, '-')

df.head()

In [ ]:
dfcolnmes.head()

In [ ]:
#clean up column names 
df.columns = df.columns.str.replace('.1','')
df.columns = df.columns.str.replace('.2','') 
df.columns = df.columns.str.replace('.3','') 
df.columns = df.columns.str.replace('.4','') 
df.columns = df.columns.str.replace('.5','')

#create 'month number' variable 
month_name = dfcolnmes.columns[0]
datetime_object = datetime.datetime.strptime(month_name, "%B")
month_num = datetime_object.month

#if month number is between 7 and 12 year1 is selected, else year2 
if 7 <= month_num <= 12:
    year_num = year1
else:
    year_num = year2

#create date string 
date = str(year_num) + '-' + str(month_num)
print(date)

#create indexes and list variable 
idx1 = [8,26,44,62,80]
idx2 = [21,39,57,75,93]
tgt = [4,22,40,58,76]

dfdurinlist=[]
for i, enum in enumerate(idx1):
    
    dfiter = df.iloc[:,[0,1,2] + list(range(enum,idx2[i]))]
    dfiter.insert(3,'Target', str(dfcolnmes.columns[tgt[i]]))
    dfiter.drop(0, inplace=True)

    #print(pd.DataFrame(dfiter.head()))
    dfiter.columns.values[6] = 'Duration: 0'
    dfiter.columns.values[7] = 'Duration: <5'
    dfiter.columns.values[8] = 'Duration: 6-10'
    dfiter.columns.values[9] = 'Duration: 11-15'
    dfiter.columns.values[10] = 'Duration: 16-20'
    dfiter.columns.values[11] = 'Duration: 20+'
    dfiter.columns.values[12] = 'Interval: 0'
    dfiter.columns.values[13] = 'Interval: 1'
    dfiter.columns.values[14] = 'Interval: 2'
    dfiter.columns.values[15] = 'Interval: 3'
    dfiter.columns.values[16] = 'Interval: 4'

    dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 94)])
    dfiter.iloc[:,0] = dfiter.iloc[:,0].fillna(method='ffill')

    #cut off dataframe rows based on number of days in month
    if month_num == 4 or month_num == 6 or month_num == 9 or month_num == 11:
        dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 90)])
    elif month_num == 2:
        dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 84)])
    else:
        dfiter = dfiter
    
    dfdurinlist.append(dfiter)

dfdurin = pd.concat(dfdurinlist)
print(dfdurin.shape)
pd.DataFrame(dfdurinlist[0].head())
dfdurin.head(100)

In [ ]:
dfdurin.drop(columns=[dfdurin.columns[4],dfdurin.columns[5]], inplace=True)
durin_m = pd.melt(dfdurin, id_vars = [dfdurin.columns[0],dfdurin.columns[1],dfdurin.columns[2],dfdurin.columns[3]])
durin_m = durin_m.groupby(['Target','variable'])['value'].sum().reset_index()
durin_m = durin_m[~durin_m['Target'].str.contains('Insert')]
durin_m.head(50) 

In [ ]:
fig = px.bar(durin_m[~durin_m['variable'].str.contains('Interval')],x = 'Target', y='value', color = 'variable',
             color_discrete_sequence= px.colors.sequential.Reds, category_orders={"variable": ["Duration: 0", "Duration: <5", "Sat", "Duration: 6-10",
                                                                                               "Duration: 11-15","Duration: 16-20", "Duration: 20+"]})
fig.update_traces(width=0.4)

fig

In [ ]:
fig = px.bar(durin_m[~durin_m['variable'].str.contains('Duration')],x = 'Target', y='value', color = 'variable',
             color_discrete_sequence= px.colors.sequential.Greens, category_orders={"variable": ["Interval: 0", "Interval: 1", "Interval: 2",
                                                                                               "Interval: 3","Duration: 4"]})
fig.update_traces(width=0.4)

fig

In [ ]:
fig = px.bar(durin_m[~durin_m['variable'].str.contains('Interval')],x = 'Target', y='value', color = 'variable',
             color_discrete_sequence= px.colors.sequential.Reds, category_orders={"variable": ["Duration: 0", "Duration: <5", "Sat", "Duration: 6-10",
                                                                                               "Duration: 11-15","Duration: 16-20", "Duration: 20+"]})
fig.update_traces(width=0.4)
fig

In [ ]:
# dfdurin1 = df.iloc[:,[0,1,2] + list(range(8,21))]
# dfdurin1.insert(3,'Target', str(dfcolnmes.columns[4]))
# dfdurin2 = df.iloc[:,[0,1,2] + list(range(26,39))]
# dfdurin2.insert(3, 'Target', str(dfcolnmes.columns[22]))
# dfdurin3 = df.iloc[:,[0,1,2] + list(range(44,57))]
# dfdurin3.insert(3, 'Target', str(dfcolnmes.columns[40]))
# dfdurin4 = df.iloc[:,[0,1,2] + list(range(62,75))]
# dfdurin4.insert(3, 'Target', str(dfcolnmes.columns[58]))
# dfdurin5 = df.iloc[:,[0,1,2] + list(range(80,93))]
# dfdurin5.insert(3, 'Target', str(dfcolnmes.columns[76]))

# dfdurin = pd.concat([dfdurin1,dfdurin2,dfdurin3,dfdurin4,dfdurin5])              
# print(dfdurin.shape)
# dfdurin2.head()

In [ ]:
#NOTE; rename 'Duration Intervals' columns with the behavior name, same as - `'Unnamed: 7':str(dfbeh.columns[3])+'_beh'`

# dfdurin = dfdurin.rename(columns = {'Unnamed: 1':'Shift','Unnamed: 2':'No Data',
#                                 'Unnamed: 11':str(dfdurin.iloc[0,7]),'Unnamed: 46':str(dfcolnmes.columns[7]),
#                                 'Unnamed: 64':str(dfcolnmes.columns[9]),'Unnamed: 79':str(dfcolnmes.columns[11])})

In [ ]:
df = pd.read_excel(wbpath, sheet_name='July', skiprows=[1,2])
dfinfo = pd.read_excel(wbpath, sheet_name=0)
dfstudet = pd.read_excel(wbpath, sheet_name=2, header=None)

#get years variable 
year = dfinfo.iloc[3,1]
year1, year2 = str.split(year, '-')

#get name
# name = dfstudet.iloc[0,0]
# print(f"name is {name}")

# pdb.set_trace()
#select necessary columns and front fill NaNs in month column 
dfbeh = df.iloc[:, [0,1,2,4,7,22,25,40,43,58,61,76,79]]
#print(dfbeh.head())
dfbeh = dfbeh.rename(columns = {'Unnamed: 1':'Shift','Unnamed: 2':'No Data','Unnamed: 7':str(dfbeh.columns[3])+'_beh','Unnamed: 25':str(dfbeh.columns[5])+'_beh',
                                'Unnamed: 43':str(dfbeh.columns[7])+'_beh','Unnamed: 61':str(dfbeh.columns[9])+'_beh','Unnamed: 79':str(dfbeh.columns[11])+'_beh'})

dfbeh.head()

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from read_workbook import *


# Create dummy data indexed by month and with multi-columns [product, revenue]
index = pd.date_range("2020", "2021", freq="MS", closed="left")
dfdur, dfin = get_all_months_int_dur("data/DP Beh Data 2022-23.xlsm")

# dfdur_wide = pd.pivot(dfdur, columns="variable", values="value")

dfdur_wide = pd.pivot_table(dfdur, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max)
dfdur_wide_grp = pd.pivot_table(dfdur_wide, index="Date", columns="variable", values=["Aggression", "Disruptive", "SIB", "Self Mutilation"])
dfdur_wide_grp.index.name = None

df = dfdur_wide_grp

/var/folders/bf/5_wbvc1n7tb3fqhf188cf6x00000gn/T/ipykernel_11342/2638987717.py:8: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  index = pd.date_range("2020", "2021", freq="MS", closed="left")


ParserError: Unknown string format: 2022-7-nan present at position 1

In [2]:
# print(df.columns)
#group data by month
df['Yr_Mnth'] = pd.to_datetime(df.index).strftime('%Y-%m')
dfm = df.groupby(['Yr_Mnth'],sort=False,)[df.columns].sum().reset_index()
df_month_data = df.groupby(['Yr_Mnth'],sort=False,)[df.columns].sum()
df = dfm.drop('Yr_Mnth', axis='columns')

In [3]:
# Create a figure with the right layout
fig = go.Figure(
    layout=go.Layout(
        height=600,
        width=1000,
        barmode="group",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
        yaxis2=go.layout.YAxis(
            visible=False,
            matches="y",
            overlaying="y",
            anchor="x",
        ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    "SIB" : {
        "Duration: 20+" : "#00D8FF",
        "Duration: 16-20": "#00BCDE",
        "Duration: 11-15": "#00A7C6",
        "Duration: 6-10": "#0089A3",
        "Duration: <5": "#006D82",
        "Duration: 0": "#005262",
    },
    "Aggression" : {
        "Duration: 20+" : "#F7FF00",
        "Duration: 16-20": "#D6DD00",
        "Duration: 11-15": "#BAC100",
        "Duration: 6-10": "#9CA200",
        "Duration: <5": "#838800",
        "Duration: 0": "#6A6E00",
    },
    "Disruptive" : {
        "Duration: 20+" : "#00FF08",
        "Duration: 16-20": "#00ED07",
        "Duration: 11-15": "#00D006",
        "Duration: 6-10": "#00AD05",
        "Duration: <5": "#008C04",
        "Duration: 0": "#006A03",
    },
    "Self Mutilation" : {
        "Duration: 20+" : "#FF0000",
        "Duration: 16-20": "#F00202",
        "Duration: 11-15": "#CA0000",
        "Duration: 6-10": "#AF0000",
        "Duration: <5": "#900000",
        "Duration: 0": "#6C0000",
    },
    #"other" : {
    #    "Duration: 20+" : "#d900ff",
    #    "Duration: 16-20": "#bd02de",
    #    "Duration: 11-15": "#9a02b5",
    #    "Duration: 6-10": "#85039c",
    #    "Duration: <5": "#68017a",
    #    "Duration: 0": "#43014f",
    #},
}

#removes all zeros rows
# df = df.loc[~(df==0).all(axis=1)]
# Add the traces
for i, t in enumerate(colors):
    base_offset = 0
    for j, col in enumerate(colors[t]):
        if (df[t][col] == 0).all():
            continue
        # print(df[t][col])
        # print(df.index.to_list())
        fig.add_trace(go.Bar(
            x=df_month_data.index,
            y=df[t][col],
            base=base_offset,
            # Set the right yaxis depending on the selected product (from enumerate)
            # yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # The values here are in milliseconds, 1billion ms is ~1/3 month
            offsetgroup=str(i),
            #offset=(i - 1) * 1,
            #width=1,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
                            )
                    )
        
        base_offset += df[t][col]
fig.show()


In [ ]:
#intensity interval data
dfin
dfin_wide = pd.pivot_table(dfin, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max)
dfin_wide

In [ ]:
        barmode="group",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
        yaxis2=go.layout.YAxis(
            visible=False,
            matches="y",
            overlaying="y",
            anchor="x",
        ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    "SIB" : {
        "Duration: 20+" : "#00D8FF",
        "Duration: 16-20": "#00BCDE",
        "Duration: 11-15": "#00A7C6",
        "Duration: 6-10": "#0089A3",
        "Duration: <5": "#006D82",
        "Duration: 0": "#005262",
    },
    "Aggression" : {
        "Duration: 20+" : "#F7FF00",
        "Duration: 16-20": "#D6DD00",
        "Duration: 11-15": "#BAC100",
        "Duration: 6-10": "#9CA200",
        "Duration: <5": "#838800",
        "Duration: 0": "#6A6E00",
    },
    "Disruptive" : {
        "Duration: 20+" : "#00FF08",
        "Duration: 16-20": "#00ED07",
        "Duration: 11-15": "#00D006",
        "Duration: 6-10": "#00AD05",
        "Duration: <5": "#008C04",
        "Duration: 0": "#006A03",
    },
    "Self Mutilation" : {
        "Duration: 20+" : "#FF0000",
        "Duration: 16-20": "#F00202",
        "Duration: 11-15": "#CA0000",
        "Duration: 6-10": "#AF0000",
        "Duration: <5": "#900000",
        "Duration: 0": "#6C0000",
    },
    #"other" : {
    #    "Duration: 20+" : "#d900ff",
    #    "Duration: 16-20": "#bd02de",
    #    "Duration: 11-15": "#9a02b5",
    #    "Duration: 6-10": "#85039c",
    #    "Duration: <5": "#68017a",
    #    "Duration: 0": "#43014f",
    #},
}

# Add the traces
for i, t in enumerate(colors):
    base_offset = 0
    for j, col in enumerate(colors[t]):
        if (df[t][col] == 0).all():
            continue
        # print(df[t][col])
        # print(df.index.to_list())
        fig.add_trace(go.Bar(
            x=df_month_data.index,
            y=df[t][col],
            base=base_offset,
            # Set the right yaxis depending on the selected product (from enumerate)
            # yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # The values here are in milliseconds, 1billion ms is ~1/3 month
            offsetgroup=str(i),
            #offset=(i - 1) * 1,
            #width=1,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
                            )
                    )
        
        base_offset += df[t][col]
fig.show()

In [ ]:
dfdur_wide = pd.pivot_table(dfdur, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max)
dfdur_wide_grp = pd.pivot_table(dfdur_wide, index="Date", columns="variable", values=["Aggression", "Disruptive", "SIB", "Self Mutilation"])
dfdur_wide_grp.index.name = None
# dfdur_wide_grp = dfdur_wide_grp.reset_index(drop=True).rename_axis(None, axis=1)
dfdur_wide_grp.head()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


# Create dummy data indexed by month and with multi-columns [product, revenue]
index = pd.date_range("2020", "2021", freq="MS", closed="left")
df = pd.concat(
    [
        pd.DataFrame(
            np.random.rand(12, 3) * 1.25 + 0.25,
            index=index,
            columns=["Revenue1", "Revenue2", "Revenue3"]
        ),
        pd.DataFrame(
            np.random.rand(12, 3) + 0.5,
            index=index,
            columns=["Revenue1", "Revenue2", "Revenue3"]
        ),
    ],
    axis=1,
    keys=["Product1", "Product2"]
)

# Create a figure with the right layout
fig = go.Figure(
    layout=go.Layout(
        height=600,
        width=1000,
        barmode="relative",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
        yaxis2=go.layout.YAxis(
            visible=False,
            matches="y",
            overlaying="y",
            anchor="x",
        ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    "Product1": {
        "Revenue1": "#F28F1D",
        "Revenue2": "#F6C619",
        "Revenue3": "#FADD75",
    },
    "Product2": {
        "Revenue1": "#2B6045",
        "Revenue2": "#5EB88A",
        "Revenue3": "#9ED4B9",
    }
}

# Add the traces
for i, t in enumerate(colors):
    print(t)
    for j, col in enumerate(df[t].columns):
        if (df[t][col] == 0).all():
            continue
        print(df[t][col])
        print(df.index)
        fig.add_trace(go.Bar(
            x=df.index,
            y=df[t][col],
            # Set the right yaxis depending on the selected product (from enumerate)
            yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # The values here are in milliseconds, 1billion ms is ~1/3 month
            offsetgroup=str(i),
            offset=(i - 1) * 1000000000,
            width=1000000000,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
                            )
        )

fig.show()
df.head()

In [ ]:
dfdur_wide = pd.pivot_table(dfdur, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max).reset_index()
dfdur_wide_grp = pd.pivot_table(dfdur_wide, index="Date", columns="variable", values=["Aggression", "Disruptive", "SIB", "Self Mutilation"])
dfdur_wide.head()

In [ ]:
dfdur_wide_grp.reset_index().head()

In [ ]:
dfdur.head(10)